In [26]:
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm, trange

storage = "/Volumes/easystore/Drones/"

cv_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cv-flight-manifest.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
per_second = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_drone-seconds-block-count.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)


per_day = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cv-blocks-drones-calls.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
race_df = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_census-race.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
poverty_df = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_acs-poverty-blockgroups.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
# flight_data = pd.read_csv("/Volumes/easystore/Drones/compiled-flight-data-gpx.csv",dtype=str)
# flight_data.head()

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [27]:
per_second["seconds"] = per_second["seconds"].astype(float)
per_second["call_count"] = per_second["call_count"].astype(float)

per_day["num_flights"] = per_day["num_flights"].astype(float)
per_day["num_calls"] = per_day["num_calls"].astype(float)

In [28]:
per_second_joined_race = pd.merge(per_second, race_df, on="GEOID20", how="left")

In [29]:
per_second_joined_race.columns

Index(['STATEFP20', 'COUNTYFP20', 'TRACTCE20', 'BLOCKCE20', 'GEOID20',
       'NAME20', 'MTFCC20', 'UR20', 'UACE20', 'UATYPE20', 'FUNCSTAT20',
       'ALAND20', 'AWATER20', 'INTPTLAT20', 'INTPTLON20', 'HOUSING20', 'POP20',
       'geometry', 'len_x', 'seconds', 'weight', 'call_count', 'Geography',
       'Geographic Area Name', 'Total', 'Total Alone', 'Total Latino',
       'Total Not Latino', 'Total White', 'Total Black or African American',
       'Total American Indian and Alaska Native', 'Total Asian',
       'Total Native Hawaiian and Other Pacific Islander',
       'Total Some Other Race', 'len_y'],
      dtype='object')

In [30]:
per_second_joined_race['population_exposure'] = (per_second_joined_race['Total'].astype(float)*per_second_joined_race['seconds'])
per_second_joined_race['population_exposure_ctrl_size'] = (per_second_joined_race['Total'].astype(float)*per_second_joined_race['seconds'])/per_second_joined_race['ALAND20'].astype(float)
per_second_joined_race['block_exposure_ctrl_size'] = (per_second_joined_race['seconds'])/per_second_joined_race['ALAND20'].astype(float)

In [42]:
per_second_joined_race

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,Total Black or African American,Total American Indian and Alaska Native,Total Asian,Total Native Hawaiian and Other Pacific Islander,Total Some Other Race,len_y,population_exposure,population_exposure_ctrl_size,block_exposure_ctrl_size,block-group-geoid
0,6,73,12600,1005,060730126001005,Block 1005,G5040,U,78661.0,U,...,0,0,0,0,0,15,0.0,0.000000,0.187882,060730126001
1,6,73,12600,1000,060730126001000,Block 1000,G5040,U,78661.0,U,...,0,0,0,0,0,15,0.0,0.000000,0.177212,060730126001
2,6,73,12600,1001,060730126001001,Block 1001,G5040,U,78661.0,U,...,0,0,0,0,0,15,0.0,0.000000,0.156135,060730126001
3,6,73,12600,1002,060730126001002,Block 1002,G5040,U,78661.0,U,...,51,1,66,0,0,15,15742925.0,151.218698,0.230868,060730126001
4,6,73,12600,1003,060730126001003,Block 1003,G5040,U,78661.0,U,...,0,0,0,1,0,15,734514.0,23.071806,0.248084,060730126001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1954,6,73,10016,1000,060730100161000,Block 1000,G5040,R,NaN,NaN,...,0,0,0,0,0,15,0.0,0.000000,0.000431,060730100161
1955,6,73,10016,1002,060730100161002,Block 1002,G5040,R,NaN,NaN,...,0,0,0,0,0,15,0.0,0.000000,0.000020,060730100161
1956,6,73,13322,3005,060730133223005,Block 3005,G5040,U,78661.0,U,...,0,0,0,0,0,15,504.0,0.081068,0.003860,060730133223
1957,6,73,13322,3000,060730133223000,Block 3000,G5040,U,78661.0,U,...,39,0,148,3,0,15,3264912.0,10.510309,0.018311,060730133223


In [45]:
analysis_df = pd.merge(per_second_joined_race,poverty_df[['% FRLP Eligible','% Below Median','block-group-geoid']],on='block-group-geoid',how='left')

In [46]:
analysis_df.to_csv('./test.csv',index=False)